# Python Intro - Final Exercise


In [1]:
import pyodbc
import pandas as pd
import numpy as np

In [2]:
### In windows, Using a ODBC DNS (predefined connection name)
##conn = pyodbc.connect('DSN=COLLEGE;Trusted_Connection=yes;')
conn=pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=EYAL-PC;"
                      "Database=COLLEGE;"
                      "Trusted_Connection=yes;")


In [3]:
courses = pd.read_sql("SELECT * FROM Courses",conn)
courses.head()
#courses.dtypes

,CourseId,CourseName,DepartmentID,TeacherId
0,1.0,English Begginers,1.0,15.0
1,2.0,Advanced English,1.0,15.0
2,3.0,Proffesional English,1.0,15.0
3,4.0,Mathematics 101,2.0,2.0
4,5.0,Matematics Advanced,2.0,13.0


In [4]:
len(courses)

31

In [5]:
classrooms = pd.read_sql("SELECT * FROM Classrooms",conn)
classrooms.head()

#classrooms.describe()
#classrooms.dtypes

,CourseId,StudentId,degree
0,10.0,1.0,86.419914
1,10.0,1.0,86.419914
2,10.0,1.0,86.419914
3,10.0,1.0,86.419914
4,11.0,1.0,79.381125


In [6]:
len(classrooms)

3016

In [7]:
departments = pd.read_sql("SELECT * FROM Departments",conn)
departments.head()
#departments.dtypes

,DepartmentId,DepartmentName
0,1.0,English
1,2.0,Science
2,3.0,Arts
3,4.0,Sport


In [8]:
len(departments)

4

In [9]:
classcourse=pd.merge(classrooms, courses, how='left', on=['CourseId'])
classcourse.head()
classcoursesep=pd.merge(classcourse,departments, how='left', left_on='DepartmentID', right_on='DepartmentId')
classcoursesep.head()
#coursedep=pd.merge(courses,departments, how='left', left_on='DepartmentID', right_on='DepartmentId')
#coursedep.head()


,CourseId,StudentId,degree,CourseName,DepartmentID,TeacherId,DepartmentId,DepartmentName
0,10.0,1.0,86.419914,Biology,2.0,3.0,2.0,Science
1,10.0,1.0,86.419914,Biology,2.0,3.0,2.0,Science
2,10.0,1.0,86.419914,Biology,2.0,3.0,2.0,Science
3,10.0,1.0,86.419914,Biology,2.0,3.0,2.0,Science
4,11.0,1.0,79.381125,Genetics,2.0,19.0,2.0,Science


In [10]:
len(classcoursesep)

3016

# Questions
 
 
## Q1. Count the number of students on each department

In [11]:
result1=classcoursesep.groupby('DepartmentName').StudentId.nunique()
print(result1)

DepartmentName
Arts       137
English     84
Science    200
Sport       81
Name: StudentId, dtype: int64


## Q2. How many students have each course of the English department and the total number of students in the department?

In [12]:
classcoursesep1=classcoursesep.copy()
classcoursesep1.head()
result2=classcoursesep1[classcoursesep1.DepartmentName=='English']
result21=result2.groupby('CourseName').StudentId.nunique()
print(result21)


CourseName
Advanced English        24
English Begginers       32
Proffesional English    38
Name: StudentId, dtype: int64


In [13]:
classcoursesep1=classcoursesep.copy()
classcoursesep1.head()
result2=classcoursesep1[classcoursesep1.DepartmentName=='English']
result22=result2.StudentId.nunique()
print(result22)


84


## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

In [14]:
classcoursesep3=classcoursesep.copy()
result3=classcoursesep3[classcoursesep3.DepartmentName=='Science']
result31=result3.groupby('CourseName').StudentId.nunique()
df3=pd.DataFrame(result31)
df3['Class_Size_Cat'] = 'Small'
df3.loc[(df3['StudentId']>22),'Class_Size_Cat']='Big'
df3.groupby('Class_Size_Cat').agg({'StudentId':['count']})


,StudentId
,count
Class_Size_Cat,
Big,10
Small,6


## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

In [15]:
students = pd.read_sql("SELECT * FROM Students",conn)
students.head()
result4=students.groupby('Gender').StudentId.nunique()
print(result4)

Gender
F    165
M    115
Name: StudentId, dtype: int64


## Q5. For which courses the percentage of male/female students is over 70%?

In [16]:
classcoursestu=pd.merge(classcourse,students, how='left', on='StudentId')
classcoursestu['F_CAT'] = 0
classcoursestu.loc[(classcoursestu['Gender']=='F'),'F_CAT']=1
result5=classcoursestu.groupby(['CourseName']).agg({"F_CAT":['count',sum,'mean']})
result5.columns = ['Count','Sum','F_Average']
result5
#result5.sort_values(by=['Average'],ascending=False)
result51=result5[result5.F_Average>0.7]
print(result51)


            Count  Sum  F_Average
CourseName                       
Sculpture      96   68   0.708333
Tenis          72   52   0.722222


In [17]:
######### שאלה 5 כתיבה נוספת הכוללת גם את מספר הקורס ###########

classcoursestu=pd.merge(classcourse,students, how='left', on='StudentId')
classcoursestu['F_CAT'] = 0
classcoursestu.loc[(classcoursestu['Gender']=='F'),'F_CAT']=1
classcoursestu.head()

result5=classcoursestu.groupby(['CourseId','CourseName']).agg({"F_CAT":['count',sum,'mean']})
result5.columns = ['Count','Sum','F_Average']
result5
result5new = result5.rename_axis(['CourseId','CourseName']).reset_index()

result5new1=result5new.loc[(result5new['F_Average']>0.7)]
result5new1


,CourseId,CourseName,Count,Sum,F_Average
21,22.0,Sculpture,96,68,0.708333
26,29.0,Tenis,72,52,0.722222


## Q6. For each department, how many students passed with a grades over 80?

In [18]:
classcoursesep6=classcoursesep.copy()
classcoursesep6.head()
result6=classcoursesep6[classcoursesep6.degree>80]
result61=result6.groupby('DepartmentName').StudentId.nunique()
print(result61)


DepartmentName
Arts        61
English     40
Science    122
Sport       39
Name: StudentId, dtype: int64


In [62]:
###### תשובה לשאלה 6 המכילה את כמות הסטודנטים עם ציון מעל 80 וסה"כ הסטודנטים #######
classcoursesep6=classcoursesep.copy()
classcoursesep6.head()
result6=classcoursesep6[classcoursesep6.degree>80]
result61=result6.groupby('DepartmentName').StudentId.nunique()
result61.columns = ['Count>80']
result61
result61new = result61.rename_axis(['DepartmentName']).reset_index()
result61new
df6=pd.merge(result61new, result1, how='left', on=['DepartmentName'])
df6.columns = ['DepartmentName', 'Grade>80', 'Total Students']
df6new=pd.DataFrame(df6)
df6new[['Grade>80', 'Total Students']] = df6new[['Grade>80', 'Total Students']].astype(int)
df6new['Percent'] =df6new['Grade>80'] / df6new['Total Students']
df6new=pd.DataFrame(test1)
df6new.dtypes
df6new

,DepartmentName,Grade>80,Total Students,Percent
0,Arts,61,137,0.445255
1,English,40,84,0.476190
2,Science,122,200,0.610000
3,Sport,39,81,0.481481


## Q7. For each department, how many students passed with a grades under 60?

In [18]:
classcoursesep7=classcoursesep.copy()
classcoursesep7.head()
result7=classcoursesep7[classcoursesep7.degree<60]
result71=result7.groupby('DepartmentName').StudentId.nunique()
print(result71)

DepartmentName
Arts       3
English    2
Science    7
Sport      1
Name: StudentId, dtype: int64


## Q8. Rate the teachers by their average student's grades (in descending order).

In [120]:
teachers = pd.read_sql("SELECT * FROM Teachers",conn)
teachers['Teach_name'] = teachers['FirstName']+" " +teachers['LastName']
teachers.head()
#len(teachers)

classcourse=pd.merge(classrooms, courses, how='left', on=['CourseId'])
classcourse.head()
classcourseteach=pd.merge(classcourse,teachers, how='left', on='TeacherId')
classcourseteach.head()
result8=classcourseteach.groupby(['Teach_name']).agg({"degree":['mean']})
result8.columns = ['AveStuGrade']
result8.sort_values(ascending=False,by='AveStuGrade')


,AveStuGrade
Teach_name,
Darick Hess,81.432703
Jordyn Baron,80.729494
Heather Metcalf,80.440242
Zachary Healy,79.365867
Kaylea Monzingo,79.365537
Daniel Bland,78.908281
Keegan Price,78.814631
Conor Mckittrick,78.699492
Maxwell Romkee,78.673509


## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

In [125]:
###    Dep-Course-Teacher-#students
classcourse=pd.merge(classrooms, courses, how='left', on=['CourseId'])
classcourse.head()
classcoursesep=pd.merge(classcourse,departments, how='left', left_on='DepartmentID', right_on='DepartmentId')
classcoursesep.head()
classcoursedepteach=pd.merge(classcoursesep,teachers, how='left', on='TeacherId')
classcoursedepteach.head()
result9=classcoursedepteach.groupby(['DepartmentName','CourseName','Teach_name']).StudentId.nunique()
df9=pd.DataFrame(result9)
#print(df9)
df9.head()



StudentId
DepartmentName CourseName      Teach_name                  
Arts           History of Arts  Caroline Knuesel         32
               Literature       Daniel Bland             39
               Musicology       Madison Batan            30
               Painting         Katerina Young           34
               Sculpture        Robert Aivazian          23

## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).

In [214]:
###    Student ID-Student NAME - # COURSE - AVE.grade by Dep.

#######   יצירת טבלה מרכזית לעבודה ########
students = pd.read_sql("SELECT * FROM Students",conn)
students['Stu_name'] = students['FirstName']+" " +students['LastName']
students.head()

classcourse=pd.merge(classrooms, courses, how='left', on=['CourseId'])
classcourse.head()
classcoursesep=pd.merge(classcourse,departments, how='left', left_on='DepartmentID', right_on='DepartmentId')
classcoursesep.head()
classcoursesdepstu=pd.merge(classcoursesep,students, how='left', on='StudentId')
classcoursesdepstu.head()

#######    יצירת טבלה על מנת לקבל לכל סטודנט ציון ממוצע כולל וחיבורה עם המרכזית ########
resultbein=classcoursesep.groupby(['StudentId']).agg({"degree":['mean']})
resultbein.columns = ['AverageDegree']
dfbein=pd.DataFrame(resultbein)
dfbein = dfbein.rename_axis(['StudentId']).reset_index()
classcoursesdepstu1=pd.merge(classcoursesdepstu,dfbein, how='left', on='StudentId')
classcoursesdepstu1.head()

#######    יצירת טבלה על מנת לקבל לכל סטודנט כמה קורסים לוקח וחיבורה עם המרכזית ########
classrooms = pd.read_sql("SELECT * FROM Classrooms",conn)
classuni=classrooms.groupby(['StudentId']).CourseId.nunique()
dfbein1=pd.DataFrame(classuni)
dfbein1 = dfbein1.rename_axis(['StudentId']).reset_index()
dfbein1.columns = ['StudentId', 'CourseCount']
classcoursesdepstu1=pd.merge(classcoursesdepstu1,dfbein1, how='left', on='StudentId')
classcoursesdepstu1.head()



#########לכל סוטנט, מחלקה, קורסים, ציון ממוצע, ציון ממוצע כללי וכמות קורסים שלוקח ######################

result10=classcoursesdepstu1.groupby(['StudentId','Stu_name','DepartmentName','CourseName','degree','AverageDegree','CourseCount']).StudentId.nunique()
df10=pd.DataFrame(result10)
df10.columns = ['Count']
df10.head()
df10 = df10.rename_axis(['StudentId','Stu_name', 'DepartmentName', 'CourseName','degree','AverageDegree','CourseCount']).reset_index()
df1010=pd.DataFrame(df10)

############### יצירת פיבוט ######################
table = pd.pivot_table(df1010, values='degree', index=['StudentId','Stu_name','CourseCount','AverageDegree'],
                    columns=['DepartmentName'], aggfunc=np.mean)
table
#########################################################################################









,,,DepartmentName,Arts,English,Science,Sport
StudentId,Stu_name,CourseCount,AverageDegree,,,,
1.0,Alexandria Heller,4,81.337515,86.677535,NaN,79.557508,NaN
2.0,Cassie Liska,5,79.137364,79.542738,NaN,78.867115,NaN
3.0,Hayley Frank,3,78.793700,NaN,83.972245,76.204427,NaN
4.0,Maxwell Kotch,3,71.481003,69.356144,70.929801,74.157066,NaN
5.0,Dylan Boehm,3,76.718650,64.494845,NaN,100.000000,65.661106
...,...,...,...,...,...,...,...
275.0,Elise Ohler,4,82.028830,94.702880,71.617451,80.897496,NaN
276.0,Dylan Bannister,3,76.784735,NaN,NaN,76.784735,NaN
277.0,Bryan Medina,6,80.482335,78.849330,NaN,81.298837,NaN
